# M1M3 actuator exploration
Craig Lage - 11-Jan-24 \
Trying to understand the different actuator forces.

In [ ]:
import sys, time, os, asyncio, copy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

## Choose an event

In [ ]:
client = EfdClient('usdf_efd')
dayObs = 20240102
seqNum = 607
eventMaker = TMAEventMaker()
event = eventMaker.getEvent(dayObs, seqNum)
start = event.begin# - TimeDelta(2.0, format='sec')
end = event.end# + TimeDelta(2.0, format='sec')
print(start.isot, end.isot)

# Now get the data

In [ ]:
#start = event.begin - TimeDelta(120.0, format='sec')
#end = event.end + TimeDelta(5.0, format='sec')
#start = end - TimeDelta(10.0, format='sec')

index = 23 # We are just looking at a single Z actuator
names = ['AccForce', 'BalForce', 'ElForce', 'AppliedForce', 'MeasuredForce', 'Error']
forceList = []
items = [f"zForces{index}", f"zForces{index}", \
         f"zForces{index}", f"zForces{index}", f"primaryCylinderForce{index}", \
         f"primaryCylinderFollowingError{index}"]
accForces = await client.select_time_series("lsst.sal.MTM1M3.appliedAccelerationForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(accForces)
balForces = await client.select_time_series("lsst.sal.MTM1M3.appliedBalanceForces", \
                                    [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(balForces)
elForces = await client.select_time_series("lsst.sal.MTM1M3.appliedElevationForces", \
                                           [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(elForces)
appForces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(appForces)
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", f"primaryCylinderForce{index}", \
                                         start, end)
forceList.append(forces)
errors = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", f"primaryCylinderFollowingError{index}", \
                                         start, end)
forceList.append(errors)

# The static force is set with a logevent when things start up
# So we have to back up 12 hours and take the most recent entry
# I checked this on Chronograf.
staticStart = start - TimeDelta(0.50, format = 'jd') 
static_forces = appForces = await client.select_time_series("lsst.sal.MTM1M3.logevent_appliedStaticForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         staticStart, end)
static_force = static_forces[items[3]].values[-1]
print(static_force)
elTMA = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['timestamp', 'actualPosition'],  start, end) 
elInc = await client.select_time_series('lsst.sal.MTM1M3.inclinometerData', \
                                            ['timestamp', 'inclinometerAngle'],  start, end) 

hp_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.hardpointActuatorData",
    ["timestamp"]
    + [f"measuredForce{hp}" for hp in range(6)]
    + [f"f{a}" for a in "xyz"]
    + [f"m{a}" for a in "xyz"],
    start,
    end,
    )

In [ ]:
ggrg_components = ['useAccelerationForces', 'useBalanceForces', 'useVelocityForces', 'triggerBoosterValves']
ggrg_start = start - TimeDelta(1.0, format = 'jd')
ggrg = await client.select_time_series('lsst.sal.MTM1M3.logevent_slewControllerSettings', \
                                        ggrg_components, ggrg_start, end)
result = ''
for component in ggrg_components:
    if ggrg[component][-1]:
        result += 'G'
    else:
        result += 'R'
print(result)

# Now plot the data

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,3,figsize=(12,5))
plt.subplots_adjust(hspace=0.8, wspace=0.3)
plt.suptitle(f"Actuator forces {dayObs} - {seqNum} - zForces{index} {result}")
counter = 0
for y in range(2):
    for x in range(3):
        ax = axs[y,x]
        ax.set_title(names[counter])
        forceList[counter][items[counter]].plot(ax=ax)
        if counter == 2:
            ax2 = ax.twinx()
            #elArc = (elTMA['actualPosition'] - elTMA['actualPosition'].values.min()) * 3600.0
            elArc = (elInc['inclinometerAngle'] - elInc['inclinometerAngle'].values.min())# * 3600.0
            #ax2.set_ylabel("Elevation change TMA (arcsec)", color='red')
            ax2.set_ylabel("Elevation change Inc. (deg)", color='red')
            #ax2.set_yticks([0,0.5, 1.0])
            #ax2.set_ylim(0,1.0)
            ax.set_ylabel("Force(N)", color='blue')
            elArc.plot(ax = ax2, color='red', ls = '--')
            #ax2.set_ylim(
        #ax.set_ylim(0, 1500)
        counter += 1
    
plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Actuator_Forces_{result}_{dayObs}_{seqNum}_{index}.png")

# The cell below checks that the applied force is the sum of the other forces.  There is some discrepancy that I don't understand.

In [ ]:
plt.suptitle(f"Applied-Acceleration-Balance-Elevation-Static {dayObs} - {seqNum} - zForces{index}")
# We have to truncate the data when the accelerationForce data stops or the arrays will
# be of unequal length.  See below.

accLength = len(forceList[0][items[0]].values)
times = copy.deepcopy(forceList[3]['timestamp'].values[0:accLength])
times -= times[0]
diff = forceList[3][items[3]].values[0:accLength] - (forceList[0][items[0]].values[0:accLength] \
                                        + forceList[1][items[1]].values[0:accLength] + forceList[2][items[2]].values[0:accLength])
diff -= static_force
plt.plot(times, diff)
plt.ylabel("Force(N)")
plt.ylim(-50,100)
plt.xlabel("Time(sec)")
plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Actuator_Force_Differences_{dayObs}_{seqNum}_{index}.png")

In [ ]:
appForces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                            ['*'], \
                                         start, end)
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", ['*'], \
                                         start, end)

In [ ]:
for column in appForces.columns:
    print(column)

In [ ]:
for column in forces.columns:
    print(column)

In [ ]:
plt.suptitle(f"Applied-Acceleration-Balance-Elevation-Static {dayObs} - {seqNum} - zForces{index}")
# We have to truncate the data when the accelerationForce data stops or the arrays will
# be of unequal length.  See below.
shift = [-.200,-.040,0]

timeList = []
fList = []
shiftedForces = []
for i in range(4):
    times = copy.deepcopy(forceList[i]['timestamp'].values)
    times -= times[0]
    timeList.append(times)
    fList.append(forceList[i][items[i]].values)
for i in range(3):
    shiftedForce = np.interp(timeList[3] + shift[i], timeList[i], fList[i])
    shiftedForces.append(shiftedForce)

diff = copy.deepcopy(fList[3])
for i in [0,1,2]:
    diff -= shiftedForces[i]

diff -= static_force
plt.plot(times, diff)
plt.plot(timeList[0], shiftedForces[0], color='green', ls='--')
plt.plot(timeList[1], shiftedForces[1], color='red', ls='--')
plt.ylim(-50,50)
plt.ylabel("Force(N)")
plt.xlabel("Time(sec)")
#plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Actuator_Force_Differences_{dayObs}_{seqNum}_{index}.png")

In [ ]:
from scipy.optimize import minimize

def func(params, args):
    #shift = [params[0], params[1], 0.0]
    shift = params    
    [timeList, fList] = args
    shiftedForces = []
    for i in range(3):
        theseForces = np.interp(timeList[3] + shift[i], timeList[i], fList[i])
        shiftedForces.append(theseForces)
    diff = copy.deepcopy(fList[3])
    for i in [0,1,2]:
        diff -= shiftedForces[i]
    diff -= static_force
    diff -= diff[-1]
    
    return np.sum(diff * diff)


In [ ]:
args = [timeList, fList]
param0 = [-.20, -.04, 0.0]
bestShift = minimize(func, param0, args=args, method='Powell')
print(bestShift.x, bestShift.fun)

In [ ]:
plt.suptitle(f"Applied-Acceleration-Balance-Elevation-Static {dayObs} - {seqNum} - zForces{index}")
# We have to truncate the data when the accelerationForce data stops or the arrays will
# be of unequal length.  See below.
shift = [bestShift.x[0], bestShift.x[1], bestShift.x[2]]

timeList = []
fList = []
shiftedForces = []
for i in range(4):
    times = copy.deepcopy(forceList[i]['timestamp'].values)
    times -= times[0]
    timeList.append(times)
    fList.append(forceList[i][items[i]].values)
for i in range(3):
    shiftedForce = np.interp(timeList[3] + shift[i], timeList[i], fList[i])
    shiftedForces.append(shiftedForce)

diff = copy.deepcopy(fList[3])
for i in [0,1,2]:
    diff -= shiftedForces[i]

diff -= static_force
plt.plot(times, diff)
#plt.plot(timeList[0], shiftedForces[0], color='green', ls='--')
#plt.plot(timeList[1], shiftedForces[1], color='red', ls='--')
plt.ylabel("Force(N)")
plt.ylim(-50,50)
plt.xlabel("Time(sec)")
#plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Actuator_Force_Differences_{dayObs}_{seqNum}_{index}.png")

# Here we check to see if there is a timebase offset between the different forces.  It is less than 1 msec.

In [ ]:
t0 = forceList[3]['timestamp'].values[0]
for i in range(4):
    print(f"Start time difference {names[i]} - {names[3]} = {(forceList[i]['timestamp'].values[0] - t0):.4f}")

# However, if we check the end times, the AccelerationForce data stops 2.7 seconds before the others.  Why is this?

In [ ]:
t0 = forceList[3]['timestamp'].values[-1]
for i in range(4):
    print(f"End time difference {names[i]} - {names[3]} = {(forceList[i]['timestamp'].values[-1] - t0):.4f}")

# We also check that all of the forces have the same 20msec cadence.

In [ ]:
t0 = forceList[3]['timestamp'].values[-1]
for i in range(4):
    deltaTs = []
    for j in range(1,6):
        deltaT = (forceList[i]['timestamp'].values[j] - forceList[i]['timestamp'].values[j-1]) * 1000.0
        deltaTs.append(f"{deltaT:.2f}")
        
    print(f"Time cadence {names[i]} = {deltaTs} msec")

In [ ]:
appForces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         start, end)
cylForces = await client.select_time_series("lsst.sal.MTM1M3.appliedCylinderForces", \
                                            [f"primaryCylinderForces{index}", 'timestamp'], \
                                         start, end)

In [ ]:
appForces[f"zForces{index}"].plot(color='green', label="appliedForces")
cylForcesN = cylForces[f"primaryCylinderForces{index}"] / 1000.0
cylForcesN.plot(color='red', ls = '--', label="appliedCylinderForces")
plt.legend()
plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Applied_vs_Cylinder_{dayObs}_{seqNum}_{index}.png")

In [ ]:
start = event.begin - TimeDelta(300.0, format='sec')
end = start + TimeDelta(60.0, format='sec')
#end = event.end + TimeDelta(300.0, format='sec')

accForces = await client.select_time_series("lsst.sal.MTM1M3.appliedAccelerationForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         start, end)
balForces = await client.select_time_series("lsst.sal.MTM1M3.appliedBalanceForces", \
                                    [f"zForces{index}", 'timestamp'], \
                                         start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['timestamp', 'actualPosition'],  start, end) 
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['timestamp', 'actualPosition'],  start, end) 


In [ ]:

dayObs = 20240108
fig, ax = plt.subplots(1,1)
ax.set_title("accelerationForces time gaps - 20230108")
accForces[f"zForces{index}"].plot(ax=ax)
ax2 = ax.twinx()
ax.set_ylabel("accForces (N)")
ax2.set_ylabel("TMA position(degrees")
el['actualPosition'].plot(ax=ax2, color='red', label="Elevation")
az['actualPosition'].plot(ax=ax2, color='green', label="Azimuth")
plt.legend()
plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/AccelerationForce_Time_Gaps_{dayObs}.png")

In [ ]:
forceTimes = accForces['timestamp'].values
for i in range(1, len(forceTimes)):
    deltaT = forceTimes[i] - forceTimes[i-1]
    if deltaT > .040:
        print(f"Gap at {Time(forceTimes[i],format='unix_tai').isot}, gap is {deltaT} seconds")

In [ ]:
forceTimes = balForces['timestamp'].values
for i in range(1, len(forceTimes)):
    deltaT = forceTimes[i] - forceTimes[i-1]
    if deltaT > .040:
        print(f"Gap at {Time(forceTimes[i],format='unix_tai').isot}, gap is {deltaT} seconds")

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(5,5))
plt.subplots_adjust(hspace=0.5)
forceList[1][items[1]].plot(ax=axs[0])
time = Time("2024-01-03T03:34:12.20", scale='utc').isot
axs[0].axvline(time,-50,25, ls='--', color='black')
for hp in range(6):
    hp_forces[f"measuredForce{hp}"].plot(ax=axs[1])
axs[1].axvline(time,-2000,2000, ls='--', color='black')